In [2]:
# import PDF
import os
import requests

# your pdf path
pdf_path = "/sample_data/(2010) Kirschenbaum - What is Digital Humanities.pdf"
print(pdf_path)

/Users/jennykwok/Library/Mobile Documents/com~apple~CloudDocs/Documents/Coding Notebook/RAG/Jenny's pipeline/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


/Users/jennykwok/Library/Mobile Documents/com~apple~CloudDocs/Documents/Coding Notebook/RAG/Jenny pipeline/(2010) Kirschenbaum - What is Digital Humanities.pdf


In [3]:
# Download PDF onine if not availale locally
if not os.path.exists(pdf_path):
    print(f"[INFO] File doesn't exist, downloading...")

    # Enter URL of PDF
    url = "https://mkirschenbaum.wordpress.com/wp-content/uploads/2011/03/ade-final.pdf"

    # the local filename to save the download file
    filename = pdf_path

    # send a GET request to the URL
    response = requests.get(url)

    # check if the request was successful

    if response.status_code == 200:
        # open the file and save it
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"[INFO] The file has been downloaded and saved as {filename}")
    else:
        print(
            f"[INFO] Failed to download the file. Status code: {response.status_code}"
        )

else:
    print(f"File {pdf_path} exists.")

File /Users/jennykwok/Library/Mobile Documents/com~apple~CloudDocs/Documents/Coding Notebook/RAG/Jenny pipeline/(2010) Kirschenbaum - What is Digital Humanities.pdf exists.


In [4]:
# Open the PDF file, see: http://github.com/pymupdf/PyMuPDF
import fitz

print("PyMuPDF is installed and the fitz module is available.")

PyMuPDF is installed and the fitz module is available.


In [5]:
from tqdm.auto import tqdm


def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    # more text formatting functions can go here
    return cleaned_text


def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append(
            {
                "page_number": page_number
                - 5,  # adjust page numbers since our PDF starts on page 42
                "page_char_count": len(text),
                "page_word_count": len(text.split(" ")),
                "page_sentence_count_raw": len(text.split(". ")),
                "page_token_count": len(text)
                / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                "text": text,
            }
        )
    return pages_and_texts


pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

/Users/jennykwok/Library/Mobile Documents/com~apple~CloudDocs/Documents/Coding Notebook/RAG/Jenny's pipeline/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
7it [00:00, 46.31it/s]


[{'page_number': -5,
  'page_char_count': 3679,
  'page_word_count': 599,
  'page_sentence_count_raw': 22,
  'page_token_count': 919.75,
  'text': 'ADE Bulletin ◆ Number 150, 2010 55 The author is associate  professor of English and  associate director of the  Maryland Institute for  Technology in the Hu\xad manities at the University  of Maryland. A version of  this article was presented  at the 2010 ADE Sum\xad mer Seminar East in  Adelphi, Maryland. People who say that the last battles of the computer revolution in En\xadglish departments have been  fought and won don’t know what they’re talking about. If our current use of computers in En\xadglish  studies is marked by any common theme at all, it is experimentation at the most basic level. As a pro\xad fession, we are just learning how to live with computers, just beginning to integrate these machines  effectively into writing- and reading-\xadintensive courses, just starting to consider the implications of the  multilayered litera

In [ ]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': -4,
  'page_char_count': 3772,
  'page_word_count': 614,
  'page_sentence_count_raw': 20,
  'page_token_count': 943.0,
  'text': 'ADE Bulletin ◆ Number 150, 2010 56 What Is Digital  Humanities and  What’s It Doing in  En\xadglish Departments? Matthew G.  Kirschenbaum strategic plan and curriculum-\xaddevelopment document, every hiring request, and so  forth that invokes the term. Or the countless times the question has been visited on  electronic discussion lists, blogs, Facebook walls, and Twitter feeds, contributing all  the flames and exhortations, celebrations and screeds one could wish to read. We could also, of course, simply Google the question. Google takes us to Wikipe\xad dia, and what we find there is not bad: The digital humanities, also known as humanities computing, is a field of study,  research, teaching, and invention concerned with the intersection of computing and  the disciplines of the humanities. It is methodological by nature and interdisciplinar

In [6]:
# reading some stats from the text
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-5,3679,599,22,919.75,"ADE Bulletin ◆ Number 150, 2010 55 The author ..."
1,-4,3772,614,20,943.00,"ADE Bulletin ◆ Number 150, 2010 56 What Is Dig..."
2,-3,3659,618,18,914.75,"ADE Bulletin ◆ Number 150, 2010 57 What Is Dig..."
3,-2,3733,617,16,933.25,"ADE Bulletin ◆ Number 150, 2010 58 What Is Dig..."
4,-1,3887,641,19,971.75,"ADE Bulletin ◆ Number 150, 2010 59 What Is Dig..."


In [7]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,7.00,7.00,7.00,7.00,7.00
mean,-2.00,3436.57,565.43,25.14,859.14
std,2.16,869.01,147.71,18.17,217.25
min,-5.00,1475.00,232.00,15.00,368.75
25%,-3.50,3669.00,606.50,17.00,917.25
50%,-2.00,3733.00,617.00,19.00,933.25
75%,-0.50,3811.50,627.50,21.00,952.88
max,1.00,3887.00,641.00,66.00,971.75


#### Note: The Importance of Token Count

It’s essential to consider token count when working with embedding models or Large Language Models (LLMs), as they cannot process an infinite number of tokens. During tokenization, some information may be lost or truncated, which can impact the quality of the embeddings and the model’s understanding of the text. Carefully managing token counts is crucial, especially for applications requiring detailed analysis, such as close reading of text.


#### Further Text Processing: Splitting Pages into Sentences

1. Splitting on "."
2. This step can be done with spaCy or nltk.


In [8]:
import sys

print(sys.executable)

/Users/jennykwok/Library/Mobile Documents/com~apple~CloudDocs/Documents/Coding Notebook/RAG/Jenny's pipeline/.venv/bin/python


Depending on your corpus, you may choose which spaCy model to use:

**Small model**
python -m spacy download en_core_web_sm

**Medium model**
python -m spacy download en_core_web_md

**Large model**
python -m spacy download en_core_web_lg


In [10]:
from spacy.lang.en import English

nlp = English()

# Add a sentencizer pipeline, c.f. https://spacy.io/api/sentencizer
nlp.add_pipe("sentencizer")

# Create document instance as an example
doc = nlp(
    "This is a sentence. This is another sentence. This is about digital humanities."
)
assert len(list(doc.sents)) == 3

# Print out sentences split
list(doc.sents)

[This is a sentence.,
 This is another sentence.,
 This is about digital humanities.]

In [11]:
pages_and_texts[1]

{'page_number': -4,
 'page_char_count': 3772,
 'page_word_count': 614,
 'page_sentence_count_raw': 20,
 'page_token_count': 943.0,
 'text': 'ADE Bulletin ◆ Number 150, 2010 56 What Is Digital  Humanities and  What’s It Doing in  En\xadglish Departments? Matthew G.  Kirschenbaum strategic plan and curriculum-\xaddevelopment document, every hiring request, and so  forth that invokes the term. Or the countless times the question has been visited on  electronic discussion lists, blogs, Facebook walls, and Twitter feeds, contributing all  the flames and exhortations, celebrations and screeds one could wish to read. We could also, of course, simply Google the question. Google takes us to Wikipe\xad dia, and what we find there is not bad: The digital humanities, also known as humanities computing, is a field of study,  research, teaching, and invention concerned with the intersection of computing and  the disciplines of the humanities. It is methodological by nature and interdisciplinary  in 

In [12]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Ensure all sentences are strings (currently spaCy datatype)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

100%|██████████| 7/7 [00:00<00:00, 69.26it/s]


In [13]:
random.sample(pages_and_texts, k=1)

NameError: name 'random' is not defined

In [14]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,7.00,7.00,7.00,7.00,7.00,7.00
mean,-2.00,3436.57,565.43,25.14,859.14,25.71
std,2.16,869.01,147.71,18.17,217.25,14.99
min,-5.00,1475.00,232.00,15.00,368.75,15.00
25%,-3.50,3669.00,606.50,17.00,917.25,19.50
50%,-2.00,3733.00,617.00,19.00,933.25,20.00
75%,-0.50,3811.50,627.50,21.00,952.88,23.50
max,1.00,3887.00,641.00,66.00,971.75,59.00


#### Chunking Text for Embedding and Retrieval

Chunking text into manageable segments is essential for effective processing in a Retrieval-Augmented Generation (RAG) pipeline.

**LangChain** provides tools to handle this chunking, making it easier to organize content to fit within the embedding model’s context window. Proper chunking ensures that the context passed to the LLM is focused and relevant, leading to more accurate and precise responses.


In [15]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10  # Experiment with the number


# Create a funcion to split lists of texts recursively into chunk size
# e.g. [20]->[10,10] or [25] -> [10,10,5]
def split_list(
    input_list: list[str], slice_size: int = num_sentence_chunk_size
) -> list[list[str]]:
    return [
        input_list[i : i + slice_size] for i in range(0, len(input_list), slice_size)
    ]


test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [16]:
# Loop throough ppers and texts and split sentencds into chunk
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(
        input_list=item["sentences"], slice_size=num_sentence_chunk_size
    )
    item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 7/7 [00:00<00:00, 40440.95it/s]


In [17]:
random.sample(pages_and_texts, k=1)

NameError: name 'random' is not defined

In [18]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,7.00,7.00,7.00,7.00,7.00,7.00,7.00
mean,-2.00,3436.57,565.43,25.14,859.14,25.71,2.86
std,2.16,869.01,147.71,18.17,217.25,14.99,1.46
min,-5.00,1475.00,232.00,15.00,368.75,15.00,2.00
25%,-3.50,3669.00,606.50,17.00,917.25,19.50,2.00
50%,-2.00,3733.00,617.00,19.00,933.25,20.00,2.00
75%,-0.50,3811.50,627.50,21.00,952.88,23.50,3.00
max,1.00,3887.00,641.00,66.00,971.75,59.00,6.00


### Splitting and Embedding Text Chunks

Each chunk of text is split into individual items and embedded as its own unique numerical representation. This process helps ensure that each segment of content is distinct and appropriately prepared for retrieval and analysis in the RAG pipeline.

The following code performs this function by:

1. **Combining Sentences into Chunks**: Each chunk is created by joining related sentences into a paragraph-like structure.
2. **Cleaning and Formatting**: Simple formatting (e.g., adding spaces after full stops) ensures consistency and readability within each chunk.
3. **Generating Statistics**: The code calculates character count, word count, and an estimated token count for each chunk, which can be useful for managing context windows and model constraints.


In [19]:
import re

# split each chunk into its own item
# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(
            r"\.([A-Z])", r". \1", joined_sentence_chunk
        )  # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(
            [word for word in joined_sentence_chunk.split(" ")]
        )
        chunk_dict["chunk_token_count"] = (
            len(joined_sentence_chunk) / 4
        )  # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# Number of chunks we have
len(pages_and_chunks)

100%|██████████| 7/7 [00:00<00:00, 3515.76it/s]


20

In [20]:
random.sample(pages_and_chunks, k=1)

NameError: name 'random' is not defined

In [21]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,20.00,20.00,20.00,20.00
mean,-1.60,1188.70,184.50,297.18
std,2.28,757.77,119.79,189.44
min,-5.00,189.00,26.00,47.25
25%,-3.25,295.75,42.25,73.94
50%,-1.50,1527.50,231.50,381.88
75%,1.00,1798.75,284.75,449.69
max,1.00,2263.00,349.00,565.75


In [22]:
df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-5,"ADE Bulletin ◆ Number 150, 2010 55 The author ...",1204,194,301.00
1,-5,"Under the earlier appellation, John Unsworth h...",1587,235,396.75
2,-5,"Crucially, there are digital humanities center...",845,129,211.25
3,-4,"ADE Bulletin ◆ Number 150, 2010 56 What Is Dig...",1471,228,367.75
4,-4,We could attempt to refine this “outlook” quan...,2263,349,565.75


In [23]:
print(df.head())  # Displays the first few rows of the DataFrame
print(
    df["chunk_token_count"].describe()
)  # Shows statistics for the chunk_token_count column

   page_number                                     sentence_chunk  \
0           -5  ADE Bulletin ◆ Number 150, 2010 55 The author ...   
1           -5  Under the earlier appellation, John Unsworth h...   
2           -5  Crucially, there are digital humanities center...   
3           -4  ADE Bulletin ◆ Number 150, 2010 56 What Is Dig...   
4           -4  We could attempt to refine this “outlook” quan...   

   chunk_char_count  chunk_word_count  chunk_token_count  
0              1204               194             301.00  
1              1587               235             396.75  
2               845               129             211.25  
3              1471               228             367.75  
4              2263               349             565.75  
count     20.00000
mean     297.17500
std      189.44172
min       47.25000
25%       73.93750
50%      381.87500
75%      449.68750
max      565.75000
Name: chunk_token_count, dtype: float64


In [24]:
# Filter chunks of text for short chunks as they may not contain much useful information
# Show random chunks with under 70 tokens in length
# You may need to adjust the number of tokens. The below code has been adjusted to check whether tokens of your expected count exists in the DatafFrame.

# Set min_token_length to filter out short chunks
min_token_length = 70

# Filter and check the DataFrame
filtered_df = df[df["chunk_token_count"] <= min_token_length]
print(
    f"Number of chunks found with token count <= {min_token_length}: {len(filtered_df)}"
)

if not filtered_df.empty:
    sample_size = min(5, len(filtered_df))
    for _, row in filtered_df.sample(sample_size).iterrows():
        print(
            f'Chunk token count: {row["chunk_token_count"]} | Text: {row["sentence_chunk"]}'
        )
else:
    print(f"No chunks found with token count <= {min_token_length}.")

Number of chunks found with token count <= 70: 5
Chunk token count: 60.5 | Text: 2 Nov. 2010. French, Amanda. “Make ‘10’ Louder; or, The Amplification of Scholarly Communication.”­Amandafrench​ .net. French, 30 Dec. 2009. Web.2 Aug. 2010. Howard, Jennifer. “The MLA Convention in Translation.”Chronicle of Higher Education.
Chunk token count: 56.75 | Text: Message to the author.5 Apr. 2010. E‑mail. ———. “What Is Humanities Computing and What Is Not?”Graduate School of Library and Information Sciences. Illinois Informatics Inst.,U of Illinois, Urbana, 8 Nov. 2002. Web.2 Aug. 2010.
Chunk token count: 54.5 | Text: Chronicle of Higher Education. Chron­ icle of Higher Educ.,28 Dec. 2009. Web.2 Aug. 2010. Selfe, Cynthia. “Computers in En­glish Departments: The Rhetoric of Technopower.”ADE Bulletin 90 (1988): 63–67. Web.2 Aug. 2010.
Chunk token count: 47.25 | Text: Chronicle of Higher Educ.,31 Dec. 2009. Web.2 Aug. 2010. McCarty, Willard. Humanities Computing. New York: Palgrave, 2005. Print. P

In [25]:
# Filter Dataframe for rows with under 70 tokens
# You will need to adjust the number of tokens depending on your DataFrame structure

pages_and_chunks_over_min_token_len = df[
    df["chunk_token_count"] > min_token_length
].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -5,
  'sentence_chunk': 'ADE Bulletin ◆ Number 150, 2010 55 The author is associate professor of English and associate director of the Maryland Institute for Technology in the Hu\xad manities at the University of Maryland. A version of this article was presented at the 2010 ADE Sum\xad mer Seminar East in Adelphi, Maryland. People who say that the last battles of the computer revolution in En\xadglish departments have been fought and won don’t know what they’re talking about. If our current use of computers in En\xadglish studies is marked by any common theme at all, it is experimentation at the most basic level. As a pro\xad fession, we are just learning how to live with computers, just beginning to integrate these machines effectively into writing- and reading-\xadintensive courses, just starting to consider the implications of the multilayered literacy associated with computers.\x08 —Cynthia Selfe What is (or are) the “digital humanities,” aka “humanities computing”

In [27]:
import random

random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': -4,
  'sentence_chunk': 'ADE Bulletin ◆ Number 150, 2010 56 What Is Digital Humanities and What’s It Doing in En\xadglish Departments?Matthew G. Kirschenbaum strategic plan and curriculum-\xaddevelopment document, every hiring request, and so forth that invokes the term. Or the countless times the question has been visited on electronic discussion lists, blogs, Facebook walls, and Twitter feeds, contributing all the flames and exhortations, celebrations and screeds one could wish to read. We could also, of course, simply Google the question. Google takes us to Wikipe\xad dia, and what we find there is not bad: The digital humanities, also known as humanities computing, is a field of study, research, teaching, and invention concerned with the intersection of computing and the disciplines of the humanities. It is methodological by nature and interdisciplinary in scope. It involves investigation, analysis, synthesis and presentation of information in electronic form. It s

#### Embedding Text Chunks

see https://vickiboykis.com/what_are_embeddings/


In [28]:
## Note: Downgraded NumPy to version <2 for compatibility with sentence-transformers.
# NumPy 2.x versions are currently not fully supported by sentence-transformers and may cause errors.
# This downgrade ensures stable integration when encoding embeddings.

import numpy as np

print("NumPy version:", np.__version__)

NumPy version: 1.26.4


In [32]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    model_name_or_path="all-mpnet-base-v2", device="cpu"
)  # Set device to "cpu" or "cuda" depending on available hardware and speed


# Create the list of sentences
sentences = [
    "The Sentence Transformer library provides an easy way to create embeddings.",
    "Sentence embedding is part of the process.",
    "This is a digital humanities project.",
]

# encode sentences by calling model.encode()
embeddings = embedding_model.encode(sentences)
embedding_dict = dict(zip(sentences, embeddings))

# check the embeddings
for sentence, embedding in embedding_dict.items():
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")
    print("")

Sentence: The Sentence Transformer library provides an easy way to create embeddings.
Embedding: [-3.44286561e-02  2.95328666e-02 -2.33643446e-02  5.57257347e-02
 -2.19098609e-02 -6.47061458e-03  1.02848643e-02 -6.57803416e-02
  2.29718257e-02 -2.61120778e-02  3.80420648e-02  5.61403073e-02
 -3.68746556e-02  1.52787985e-02  4.37020436e-02 -5.19723631e-02
  4.89479713e-02  3.58103635e-03 -1.29750641e-02  3.54384817e-03
  4.23262678e-02  3.52606401e-02  2.49402598e-02  2.99176928e-02
 -1.99382566e-02 -2.39752196e-02 -3.33364634e-03 -4.30450402e-02
  5.72014488e-02 -1.32517964e-02 -3.54478136e-02 -1.13936029e-02
  5.55561408e-02  3.61095602e-03  8.88526756e-07  1.14027224e-02
 -3.82229947e-02 -2.43544439e-03  1.51314372e-02 -1.32716275e-04
  5.00659943e-02 -5.50876446e-02  1.73444748e-02  5.00959530e-02
 -3.75959128e-02 -1.04463520e-02  5.08322231e-02  1.24861291e-02
  8.67377520e-02  4.64142784e-02 -2.10690200e-02 -3.90251316e-02
  1.99694769e-03 -1.42345643e-02 -1.86794661e-02  2.826696

In [33]:
embeddings[0].shape

(768,)

In [35]:
import time
from tqdm import tqdm

# Start the timer
start_time = time.time()

# Move the model to the CPU for embedding creation
embedding_model.to("cpu")

# Generate embeddings for each chunk on the CPU
# This iterates over each item in pages_and_chunks_over_min_token_len
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

# Calculate and print the elapsed time
elapsed_time = time.time() - start_time
print(f"Time taken for CPU embedding creation: {elapsed_time:.2f} seconds")

# Optional: Move the model to the GPU for faster embedding creation
# Uncomment the following lines if a GPU is available for use with "cuda"

# embedding_model.to("cuda")  # Requires a GPU to be installed

# # Start timing for GPU embedding
# start_time = time.time()

# # Create embeddings on the GPU (if available)
# for item in tqdm(pages_and_chunks_over_min_token_len):
#     item["embedding"] = embedding_model.encode(item["sentence_chunk"])

# # Calculate and print the elapsed time for GPU processing
# elapsed_time = time.time() - start_time
# print(f"Time taken for GPU embedding creation: {elapsed_time:.2f} seconds")

100%|██████████| 15/15 [00:05<00:00,  2.59it/s]

Time taken for CPU embedding creation: 5.80 seconds


In [37]:
import pandas as pd

# Measure execution time for batch embedding and saving (if in Jupyter, uncomment %%time)
# %%time

# Embed all text chunks in batches to optimize memory and performance
# Uncomment the following to batch embeddings:
# text_chunk_embeddings = embedding_model.encode(
#     text_chunks,  # List of text chunks to embed
#     batch_size=32,  # Adjust batch size as needed to improve speed and memory efficiency
#     convert_to_tensor=True  # Optional: return embeddings as PyTorch tensor for further tensor operations
# )

# Optional: Display or use `text_chunk_embeddings` for further processing

# Save the text chunks and embeddings to a DataFrame
# Note: Adjust path as needed
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(
    embeddings_df_save_path, index=False
)  # Save DataFrame as CSV

In [38]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-5,"ADE Bulletin ◆ Number 150, 2010 55 The author ...",1204,194,301.00,[ 1.79897361e-02 3.91299501e-02 -4.22144756e-...
1,-5,"Under the earlier appellation, John Unsworth h...",1587,235,396.75,[ 4.28221188e-02 3.57256345e-02 -4.90087867e-...
2,-5,"Crucially, there are digital humanities center...",845,129,211.25,[ 2.03686710e-02 3.22138183e-02 -2.67556552e-...
3,-4,"ADE Bulletin ◆ Number 150, 2010 56 What Is Dig...",1471,228,367.75,[ 2.54732836e-02 2.58226339e-02 -3.68595086e-...
4,-4,We could attempt to refine this “outlook” quan...,2263,349,565.75,[ 4.28117551e-02 4.96095791e-02 -1.71628781e-...


### Query


In [40]:
import random
import torch
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
    "embedding"
].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(
    np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32
).to(device)
embeddings.shape
embeddings.dtype

torch.float32

In [41]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-5,"ADE Bulletin ◆ Number 150, 2010 55 The author ...",1204,194,301.00,"[0.0179897361, 0.0391299501, -0.0422144756, -0..."
1,-5,"Under the earlier appellation, John Unsworth h...",1587,235,396.75,"[0.0428221188, 0.0357256345, -0.0490087867, 0...."
2,-5,"Crucially, there are digital humanities center...",845,129,211.25,"[0.020368671, 0.0322138183, -0.0267556552, 0.0..."
3,-4,"ADE Bulletin ◆ Number 150, 2010 56 What Is Dig...",1471,228,367.75,"[0.0254732836, 0.0258226339, -0.0368595086, -0..."
4,-4,We could attempt to refine this “outlook” quan...,2263,349,565.75,"[0.0428117551, 0.0496095791, -0.0171628781, -0..."


In [42]:
text_chunks_and_embedding_df["embedding"]

0     [0.0179897361, 0.0391299501, -0.0422144756, -0...
1     [0.0428221188, 0.0357256345, -0.0490087867, 0....
2     [0.020368671, 0.0322138183, -0.0267556552, 0.0...
3     [0.0254732836, 0.0258226339, -0.0368595086, -0...
4     [0.0428117551, 0.0496095791, -0.0171628781, -0...
5     [0.0407331511, 0.0463119335, -0.0121963033, -0...
6     [0.0357671082, 0.0379991904, -0.0240194853, -0...
7     [0.0288532246, 0.0799177364, -0.0226198602, -0...
8     [0.0360140726, 0.0506579988, -0.0104648927, -0...
9     [0.0773864239, 0.0159307439, -0.0220139213, -0...
10    [0.0680856034, 0.0339299589, -0.0140206954, -0...
11    [0.0413060784, 0.0123664839, 0.00236883759, -0...
12    [0.0459209159, 0.0191587787, -0.0439116918, -0...
13    [0.0413302518, 0.0325960033, -0.00894801784, -...
14    [0.018890053, 0.0348874405, -0.0441482589, -0....
Name: embedding, dtype: object

In [43]:
embeddings = np.stack(text_chunks_and_embedding_df["embedding"].tolist(), axis=0)
embeddings

array([[ 0.01798974,  0.03912995, -0.04221448, ...,  0.03801661,
         0.01319847, -0.04493817],
       [ 0.04282212,  0.03572563, -0.04900879, ...,  0.03242273,
         0.02076461, -0.01721456],
       [ 0.02036867,  0.03221382, -0.02675566, ...,  0.02016196,
         0.01865645, -0.01403168],
       ...,
       [ 0.04592092,  0.01915878, -0.04391169, ...,  0.01148756,
        -0.01844093, -0.0394499 ],
       [ 0.04133025,  0.032596  , -0.00894802, ..., -0.00340151,
         0.03831667, -0.0250778 ],
       [ 0.01889005,  0.03488744, -0.04414826, ...,  0.00217194,
         0.03867229, -0.04262922]])

In [44]:
embeddings.shape

(15, 768)

In [45]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(
    model_name_or_path="all-mpnet-base-v2", device=device
)  # choose the device to load the model to

### Steps for Querying with Embeddings

1. **Define a Query String**: Start by creating a query as a text string that represents what you’re searching for.

2. **Convert the Query to an Embedding**: Use the same embedding model to transform the query string into an embedding, similar to how the text chunks were embedded.

3. **Calculate Similarity**: Perform a similarity comparison (e.g., dot product or cosine similarity) between the query embedding and each text embedding in the dataset.

4. **Sort Results by Relevance**: Sort the similarity scores in descending order to identify the most relevant results for your query.


In [48]:
import torch
from sentence_transformers import util

# 1. Define the query
query = "digital humanities"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
# Ensure that both query_embedding and embeddings are in the same format and dtype
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# Convert embeddings to a PyTorch tensor if it's currently a NumPy array
if isinstance(embeddings, np.ndarray):
    embeddings = torch.tensor(embeddings, dtype=torch.float32)

# 3. Get similarity scores with the dot product
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(
    f"Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds."
)

# 4. Get the top-k results (keeping this to 5 for now)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: digital humanities
Time taken to get scores on 15 embeddings: 0.00015 seconds.


torch.return_types.topk(
values=tensor([0.8262, 0.8018, 0.7910, 0.7887, 0.7742]),
indices=tensor([ 3, 14,  1,  4,  2]))

In [55]:
# Query the output by inputting a number from the indices
pages_and_chunks[2]

{'page_number': -5,
 'sentence_chunk': 'Crucially, there are digital humanities centers and institutes (probably at least one hundred worldwide, some of them estab\xad lished for a decade or more with staffs numbering in the dozens): these are served by an organization known as centerNet. There have been digital humanities manifestos (I know of at least two) and FAQs, colloquia and symposia, workshops and special sessions. Not to mention, of course, that a gloss or explanation of digital humani\xad ties is implicit in every mission statement, every call for papers and proposals, every What Is Digital Humanities and What’s It Doing in English Departments?Matthew G. Kirschenbaum ADE and the Association of Departments of En\xadglish are trademarks owned by the Modern Language Association. © by the Association of Departments of En\xadglish, CrossRef DOI: 10.1632/ade.150.55, ISSN 0001-0898',
 'chunk_char_count': 845,
 'chunk_word_count': 129,
 'chunk_token_count': 211.25,
 'embedding': arra

In [54]:
larger_embeddings = torch.randn(100 * embeddings.shape[0], 768).to(device)
print(f"Embeddings shape:  {larger_embeddings.shape}")

# Perform dot product across 168,000 embeddings
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
embeddings.dtype
end_time = timer()

print(
    f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds."
)

Embeddings shape:  torch.Size([1500, 768])
Time take to get scores on 1500 embeddings: 0.00132 seconds.
